# Markovian simulation

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dx = [1./10,1./10]
nx = [300,300]

In [ ]:
db = gl.DbGrid.create(nx, dx)

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt
scale = 1
mesh = gl.MeshETurbo(nx,dx)
model = gl.Model.createFromParam(gl.ECov.MARKOV,range=1.,sill=1,flagRange=False)
#model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,range=scale,sill=1,flagRange=False)

cova = model.getCova(0)
cova.setMarkovCoeffs([1,-0.3*scale,-1.5*scale**2,2*scale**3,0,1*scale**5])
Q = gl.PrecisionOp(mesh,model)


In [ ]:
indx = int(nx[0]/2)
indy = int(nx[1]/2)
ind = indy + nx[1] * indx
covy = np.array(Q.evalCov(ind)).reshape(nx)
plt.plot(covy[indx,indy:(indy+nx[1])])

In [ ]:
dbg = gl.DbGrid.create(nx,dx)
x = Q.simulate()[0]
dbg["simu"]=x
gp.plot(dbg)

In [ ]:
N = 2**8
nx1 = 20
nx2 = 30
result = np.array(cova.evalCovFFT([nx1,nx2],N).getValues())
res = np.array(result).reshape((N,N))

mH1 = nx1
mH2 = nx2

X1 = np.linspace(-mH1,mH1,N) 
X2 = np.linspace(-mH2,mH2,N) 
plt.imshow(res,extent = [-nx1,nx1,-nx2,nx2])
plt.colorbar()

In [ ]:
plt.plot(X1 , res[int(N/2),:],label="FFT")

uu = covy[indx,indy:(indy+100)]
plt.plot(np.arange(len(uu))*dx[0],uu,"--",label = "Discretized covariance")
ax = plt.legend()